# Etapa 2

A partir do arquivo que está dentro do S3, crie um dataframe com pandas ou polars e execute as análises escolhidas.

In [ ]:
# Importação das bibliotecas necessárias
import boto3
import pandas as pd

In [ ]:
# Importando o arquivo
arquivo = pd.read_csv('arp-2022-2023.csv')
arquivo.drop_duplicates()

In [ ]:
# Tratamento dos dados

# Tratando valores numéricos
arquivo['valor'] = arquivo['valor'].apply(lambda x: x.replace('R$ ', ''))
arquivo['valor'] = arquivo['valor'].apply(lambda x: x.replace('.', ''))
arquivo['valor'] = arquivo['valor'].astype(float)

# Tratando as datas
arquivo['fim_vigencia'] = arquivo['fim_vigencia'].apply(lambda x: x if x != 'não informado' else '')
arquivo['fim_vigencia'] = pd.to_datetime(arquivo['fim_vigencia'], format="mixed")
arquivo['fim_vigencia'] = arquivo['fim_vigencia'] + pd.DateOffset(years=1)

arquivo['inicio_vigencia'] = arquivo['inicio_vigencia'].apply(lambda x: x if x != 'NÃO CONSTA DATA' else None)
arquivo['inicio_vigencia'] = pd.to_datetime(arquivo['inicio_vigencia'], format="mixed")

# Removendo colunas desnecessárias
arquivo = arquivo[['pregão', 'ano', 'número', 'cpf_cnpj', 'fornecedor', 'objeto', 'valor', 'assinatura', 'inicio_vigencia', 'fim_vigencia']]

## Análise/Questionamento 1

Contratos de Medicamentos Humanos com Valor Superior a R$ 20.000 em 2023

In [ ]:
# Filtragem dos dados
dados_filtrados = arquivo.loc[
    (arquivo['objeto'] == 'MEDICAMENTO HUMANO') \
    & (arquivo['ano'] == 2023) \
    & (arquivo['valor'] >= 20000)
]

# Exportação dos dados
dados_filtrados.to_csv('analise_1.csv', index=False)

## Análise/Questionamento 2

Valor Total de Contratos por Tipo de Objeto e Duração Média da Vigência

In [ ]:
# Calcular duração da vigência em dias
arquivo['duracao_vigencia'] = (arquivo['fim_vigencia'] - arquivo['inicio_vigencia']).dt.days

# Agregar valor total por objeto e duração média
analise = arquivo.groupby('objeto').agg(
    valor_total=('valor', 'sum'),
    duracao_media=('duracao_vigencia', 'mean')
).reset_index()

# Exportação dos dados
analise.to_csv('analise_2.csv', index=False)

## Análise/Questionamento 3

Classificar Fornecedores "LTDA" com Base no Valor do Contrato

In [ ]:
# Classificar valores em categorias
condicoes = [
    arquivo['valor'] > 100000,
    (arquivo['valor'] >= 50000) & (arquivo['valor'] <= 100000),
    arquivo['valor'] < 50000
]
categorias = ['Alto', 'Médio', 'Baixo']

arquivo['categoria_valor'] = pd.cut(arquivo['valor'], bins=[0, 50000, 100000, float('inf')], labels=categorias)

# Filtrando apenas os fornecedores LTDA
empresas_ltda = arquivo.loc[arquivo['fornecedor'].str.contains('LTDA')]
empresas_ltda = empresas_ltda[['fornecedor', 'valor', 'categoria_valor']]

# Exportação dos dados
empresas_ltda.to_csv('analise_3.csv', index=False)